# Large Language Model (LLM) Inference Testing

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

---

## Overview 

This notebook is for real-time Large Language Model (LLM) testing in SageMaker. This notebook supports [Deep Java Library (djl) Large Model Inference (LMI)](https://docs.djl.ai/docs/serving/serving/docs/lmi/conceptual_guide/lmi_engine.html) and [Triton Inference Server](https://developer.nvidia.com/triton-inference-server) to deploy the models in SageMaker.

This notebook is driven by [YAML recipe files](#yaml-recipe-ile). The `test` object in the YAML recipe file defines the testing module interface. The testing module specifies a prompt generator class that yields prompts for the tests. This notebook supports both Unit Testing, and load testing using open-source [Locust](https://locust.io/).  

The key purpose of this notebook is to explore the inference engine configuration space to find optimal configuration for a given use case. Example configurations are included under the `examples` folder. You should use this notebook to try different configuration settings, and find the optimal settings for your use case.


## Prerequisites

This notebook should be run in an [Amazon SageMaker Notebook](https://aws.amazon.com/sagemaker-ai/notebooks/) using `ml.m5.2xlarge`, or larger, instance.

Since we are working with LLM inference, we will need to download HuggingFace model snapshots, and upload them to S3 bucket. We may also need to build Docker containers. To be able to do these actions without running out of disk space, configure at least 500 GB (1000 GB recommended) volume with the SageMaker Notebook instance, and configure this volume to be used as Docker overlay file-system by executing following steps:

#### Create new Docker overlay file-system directory:

    sudo mkdir -p /home/ec2-user/SageMaker/docker-overlay
    sudo chown root:root /home/ec2-user/SageMaker/docker-overlay

#### Update the Docker overlay file-system configuration 

Add following content to `/etc/docker/daemon.json`

    {
      "data-root": "/home/ec2-user/SageMaker/docker-overlay",
      "runtimes": {
          "nvidia": {
              "path": "nvidia-container-runtime",
              "runtimeArgs": []
          }
      }
    }

#### Restart Docker daemon

    sudo systemctl stop docker
    sudo systemctl start docker
    sudo systemctl status docker.service


## YAML Recipe File

To deploy a model, you need to define a [YAML recipe file](./examples/vllm/meta-llama3-8b-instruct/config.yaml). Below, we provide a brief explanation for the syntax of the YAML recipe:

* The `huggingface` object is optional: 
    * If you specify `huggingface` object, the `huggingface.name` field is required
    * The `huggingface.revision` is required if `huggingface.download` is `true`.
    * if `huggingface` is specified, `djl` object must be specified
* The `djl` object is optional and if specified contains the content for [DJL serving.properties](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html) for the selected [LMI engine](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/index.html)
    * If you specify `huggingface` object, `djl.option\.model_id` is computed automatically.
* The `sagemaker.model` object is required 
    * The field `sagemaker.model.image` is optional: Alternatively, you can specify `sagemaker.model.container`  as relative path (w.r.t. to this notebook) to the container build script directory.
    * The field `sagemaker.model.env` is optional
* The `sagemaker.endpoint` object is required
    * In `sagemaker.endpoint` object, only `name` and `instance_type` are required.
* The `test` object is optional, and defines the test interface 
    * The `test.module_dir` is relative path. This path is added to `sys.path`. If there is a `requirements.txt` in this directory, it is installed.
    * The `test.module_name` must be a Python module in `test.module_dir`. This module is dynamically loaded.
    * The `test.prompt_generator` is the name of a class in the `test.module_name` module. An object of this class is dyamically created, and the `__call__` method on the object is called to get the prompt generator for testing.
    * The `test.warmpup_iters` are used to warmup the deployed inference model.
    * The `n_concurrent` sepecifies number of concurrent processes used in Unit Testing.
    * The `test.max_iters` limits the number of prompt requests, not including the `test.warmup_iters`.
    * The `test.output_dir` is the relative path to the testing output root directory. 
    * The `test.locust_users` and `test.locust_workers` fields specify Locust Users and Workers, respectively, used in Locust Testing.
    * The `test.template` and `test.template_keys` fields specify the structure of the prompt template. The keys are used to update the template with prompt values.
    
### Custom Model Handler Code

You can optionally add a [custom model handler](https://github.com/deepjavalibrary/djl-serving/blob/bc7fdfdcbb66b982522e6bc809b0044fabde69e0/serving/docs/streaming_config.md#custom-modelpy-handler) in the `code` sub-folder, collocated with the LMI configuration file, and it is added to the SageMaker model package.

## Initialize Notebook

### Install Required Packages

Next, install latest version of required packages.

In [ ]:
!pip3 install sagemaker

### Initialize SageMaker Session

Below, we initialize the SageMaker session. If no `s3_bucket` is set below, the default SageMaker bucket in the region is used.Specify below the optional `s3_bucket` and `s3_prefix` that we will use throughout the notebook.

In [ ]:
import boto3
import sagemaker

s3_bucket = None  # specify bucket, or use default sagemaker bucket, if it exists
s3_prefix = "lmi-djl"  # Large model inference with deep java library

role = sagemaker.get_execution_role()  # you may provide a pre-existing role ARN here
print(f"SageMaker Execution Role: {role}")

session = boto3.session.Session()
aws_region = session.region_name
print(f"AWS Region: {aws_region}")

sagemaker_session = sagemaker.session.Session(boto_session=session)

try:
    if s3_bucket is None:
        s3_bucket = sagemaker_session.default_bucket()
    s3_client = boto3.client("s3")
    response = s3_client.get_bucket_location(Bucket=s3_bucket)
    bucket_region = response["LocationConstraint"]
    bucket_region = "us-east-1" if bucket_region is None else bucket_region

    print(f"Bucket region: {bucket_region}")

    try:
        s3_client.head_object(Bucket=s3_bucket, Key=f"{s3_prefix}/")
    except:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_prefix}/")

    print(f"Using S3 folder: s3://{s3_bucket}/{s3_prefix}/ in this notebook")
except:
    print(
        f"Access Error: Check if '{s3_bucket}' S3 bucket is in '{aws_region}' region, and {s3_prefix} path exists"
    )

sts = boto3.client("sts")
aws_account_id = sts.get_caller_identity()["Account"]

print(f"AWS Account Id: {aws_account_id}")

## Specify YAML Recipe

We specify a YAML file for the model we wish to deploy and test. For example, we specify [vLLM LMI configuration file for Llama3 8B below](./examples/vllm/meta-llama3-8b-instruct/config.yaml), below. You may change `config_path` below to the *relative path* of `examples` YAML file you wish to deploy.

In [ ]:
import os
import yaml
import json

import pathlib
from utils import install_pip_requirements, install_pip_package

print(f"Current working directory: {pathlib.Path().resolve()}")

config_path = "examples/vllm/meta-llama3-8b-instruct/config.yaml"
with open(config_path, "r") as mf:
    model_config = yaml.safe_load(mf)

print("\nmodel_config:\n")
print(json.dumps(model_config, indent=2))

assert (
    model_config.get("huggingface", None) is None or model_config.get("djl", None) is not None
), "'djl' must be specified if 'huggingface' is specified"

assert model_config.get("sagemaker", None), "'sagemaker' object is required"
assert model_config["sagemaker"].get("model", None), "'sagemaker.model' is required"
assert model_config["sagemaker"].get("endpoint", None), "'sagemaker.endpoint' is required"

test_spec = model_config.get("test", None)
if test_spec:
    requirements_path = os.path.join(os.path.dirname(config_path), "requirements.txt")
    if os.path.isfile(requirements_path):
        install_pip_requirements(requirements_path)
    else:
        install_pip_package(package_name="transformers")

## Maybe Build and Push Model Inference Container Image to ECR

Next, if `sagemaker.model.container` is specified, we build and push the container image to Amazon ECR. After building and pushing the image to ECR, we update the `sagemaker.model.image` with the ECR URI for the image. Either `container` or `image` must be specified in `sagemaker.model`.

In [ ]:
import os, stat, re
from utils import push_ecr_container

sm_model_config = model_config["sagemaker"]["model"]
container_path = sm_model_config.get("container", None)

if container_path is not None:
    sm_model_config["image"] = push_ecr_container(
        container_path=container_path, aws_region=aws_region, aws_account_id=aws_account_id
    )
else:
    ecr_image_uri = sm_model_config.get("image", None)
    assert (
        ecr_image_uri is not None
    ), "'sagemaker.model.image' or 'sagemaker.model.container' is required"
    pattern = "\.dkr\.ecr\.[a-z0-9-]+\."
    replace = f".dkr.ecr.{aws_region}."
    sm_model_config["image"] = re.sub(pattern, replace, ecr_image_uri)

print(f"Model serving image: {sm_model_config['image']}")

## Maybe Download HuggingFace Model Snapshot

If `huggingface` object is specified, only `huggingface.model` is required. If `huggingface.download` is `true`, the HuggingFace model snapshot with revision `huggingface.revision` is downloaded and uploaded to the `s3_bucket`, and `djl.option\.model_id` is set to the S3 URI of the uploaded HuggingFace model snapshot. 

**Note: You must specify `hf_token` below if the Hugging Face model requires a Hugging Face token for downloading the model from the HuggingFace hub. This is true whether or not you specify the `huggingface` object.**

In [ ]:
from utils import snapshot_hf_model_to_s3

hf_token = None  # Specify HuggingFace token, if required to access model
hf_spec = model_config.get("huggingface", None)

if hf_spec:
    snapshot_hf_model_to_s3(
        s3_client,
        s3_prefix=s3_prefix,
        s3_bucket=s3_bucket,
        model_config=model_config,
        hf_spec=hf_spec,
        hf_token=hf_token,
    )

# Create SageMaker Model Package

Next, we create the model package TAR ball, and upload it to `s3_bucket`. The model package will be used to define the SageMaker model.

In [ ]:
from utils import djl_model_package_to_s3
from utils import triton_model_package_to_s3

sm_model_name = sm_model_config.get("name", None)
assert sm_model_name, "'sagemaker.model.name' is required"
djl_spec = model_config.get("djl", None)
tritonserver = sm_model_config.get("tritonserver", None)

model_pkg_key = None
if djl_spec is not None:
    model_pkg_key = djl_model_package_to_s3(
        s3_client,
        djl_spec=djl_spec,
        sm_model_name=sm_model_name,
        config_path=config_path,
        s3_prefix=s3_prefix,
        s3_bucket=s3_bucket,
    )
elif tritonserver is not None:
    model_pkg_key = triton_model_package_to_s3(
        s3_client,
        sm_model_name=sm_model_name,
        config_path=config_path,
        s3_prefix=s3_prefix,
        s3_bucket=s3_bucket,
    )

assert model_pkg_key, "Model package key is not set"

## Create SageMaker Model

Next, we create the SageMaker model using the model package we just uploaded tp `s3_bucket`.

In [ ]:
import boto3

sm_client = boto3.client("sagemaker")
sm_model_image = sm_model_config.get("image", None)
assert sm_model_image, "'sagemaker.model.image' is required"

sm_model_env = sm_model_config.get("env", {})
if (tritonserver or sm_model_env.get("HF_MODEL_ID", None) is not None) and hf_token is not None:
    sm_model_env["HF_TOKEN"] = hf_token

primary_container = {"Image": sm_model_image, "Environment": sm_model_env}
if model_pkg_key is not None:
    primary_container["ModelDataUrl"] = f"s3://{s3_bucket}/{model_pkg_key}"

try:
    create_model_response = sm_client.create_model(
        ModelName=sm_model_name,
        ExecutionRoleArn=role,
        PrimaryContainer=primary_container,
    )
    model_arn = create_model_response["ModelArn"]

    print(f"Created Model: {model_arn}")
except Exception as e:
    print(f"Error creating model: {e}")

## Create SageMaker Endpoint Config

Next we create endpoint config for the SageMaker model. 

In [ ]:
sm_endpoint_spec = model_config["sagemaker"]["endpoint"]
endpoint_name = sm_endpoint_spec.get("name", None)
assert endpoint_name, "'sagemaker.endpoint.name' is required"

variant_name = sm_endpoint_spec.get("variant_name", "test")

instance_type = sm_endpoint_spec.get("instance_type", None)
assert instance_type, "'sagemaker.endpoint.instance_type' is required"

initial_instance_count = sm_endpoint_spec.get("initial_instance_count", 1)
model_data_download_timeout_secs = sm_endpoint_spec.get("model_data_download_timeout_secs", 1200)
container_startup_health_check_timeout_secs = sm_endpoint_spec.get(
    "container_startup_health_check_timeout_secs", 1200
)

production_variant = {
    "VariantName": variant_name,
    "ModelName": sm_model_name,
    "InstanceType": instance_type,
    "InitialInstanceCount": initial_instance_count,
    "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_secs,
    "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_secs,
    "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
}

volume_size_gb = sm_endpoint_spec.get("volume_size_gb", None)
if volume_size_gb:
    production_variant["VolumeSizeInGB"] = volume_size_gb

try:
    endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_name, ProductionVariants=[production_variant]
    )
    print(endpoint_config_response)
except Exception as e:
    print(f"Error creating endpoint config: {e}")

## Create SageMaker Endpoint

Next, we create the SageMaker Endpoint.

In [ ]:
import time

try:
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpoint_name, EndpointConfigName=endpoint_name
    )
    print_counter = 0
    while True:
        response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = response["EndpointStatus"]
        if status == "InService":
            print(f"Endpoint {endpoint_name} is ready!")
            break
        elif status == "Failed":
            print(f"Endpoint creation failed: {response['FailureReason']}")
            break
        if print_counter % 10 == 0:
            print(f"Endpoint status: {status}...")
        print_counter += 1
        time.sleep(30)  # Wait for 30 seconds before checking again
except Exception as e:
    print(f"Error creating endpoint: {e}")

## Run Unit Testing

Now, we are ready to run our unit tests, using the test interface. During this step, we download model tokenizer configuration files, and create a tokenizer. Next, we dynamically load the test interface Python module, create a prompt generator class object, and use it to drive our unit test run.

In [ ]:
if test_spec:
    from test_task import test_task

    prompt_module_dir = test_spec.get("module_dir", None)
    assert prompt_module_dir, "'test.module_dir' is required"
    requirements_path = os.path.join(prompt_module_dir, "requirements.txt")
    if os.path.isfile(requirements_path):
        install_pip_requirements(requirements_path)

    sm_endpoint_spec = model_config["sagemaker"]["endpoint"]
    endpoint_name = sm_endpoint_spec.get("name", None)
    assert endpoint_name, "'sagemaker.endpoint.name' is required"

    instance_type = sm_endpoint_spec.get("instance_type", None)
    assert instance_type, "'sagemaker.endpoint.instance_type' is required"

    seq_len_options = [
        "option.max_tokens",
        "option.n_positions",
        "option.max_model_len",
        "option.max_num_tokens",
    ]
    seq_len = 0

    if djl_spec is not None:
        for seq_len_option in seq_len_options:
            seq_len = djl_spec.get(seq_len_option, 0)
            if seq_len:
                break

    tp_degree = djl_spec.get("option.tensor_parallel_degree", 1) if djl_spec is not None else 1
    rbs = djl_spec.get("option.max_rolling_batch_size", 1) if djl_spec is not None else 1

    instance_count = sm_endpoint_spec.get("initial_instance_count", 1)

    output_dir = test_spec.get("output_dir", None)
    assert output_dir, "'test.output_dir' is required"
    os.makedirs(output_dir, exist_ok=True)
    results_path = os.path.join(
        output_dir,
        "unit-testing",
        sm_model_name,
        f"type={instance_type}",
        f"count={instance_count}",
        f"slen={seq_len}",
        f"tp={tp_degree}",
        f"rbs={rbs}",
    )

    print(f"Unit results path: {results_path}")

    output_formatter = djl_spec.get("option.output_formatter", None) if djl_spec else None
    rolling_batch = djl_spec.get("option.rolling_batch", None) if djl_spec else None
    streaming_enabled = rolling_batch in [
        "auto",
        "deepspeed",
        "trtllm",
        "vllm",
    ] and output_formatter in ["jsonlines"]
    print(f"Streaming enabled: {streaming_enabled}")

    model_id = (
        djl_spec.get("option.model_id")
        if djl_spec
        else sm_model_env.get("MODEL_ID", sm_model_env.get("HF_MODEL_ID"))
    )
    test_task(
        model_id=model_id,
        test_spec=test_spec,
        endpoint_name=endpoint_name,
        results_path=results_path,
        streaming_enabled=streaming_enabled,
        hf_token=hf_token,
    )

    print(f"Uploading unit results to S3 bucket...")
    for root, dirs, files in os.walk(results_path):
        for file in files:
            full_path = os.path.join(root, file)
            with open(full_path, "rb") as data:
                s3_client.upload_fileobj(data, s3_bucket, full_path)
    print(f"Uploading unit results to S3 bucket completed.")

## Visualize Unit Test Results

Below, we visualize the unit test results to make sure the model output is as expected. This allows us to verify that our endpoint was deployed correctly, and that there are no configuration errors. 


In [ ]:
import os
import glob
import pandas as pd
from IPython.display import display

if test_spec:
    results_files = glob.glob(os.path.join(results_path, "*.json"))
    result_latest = max(results_files, key=os.path.getmtime)

    result_latest_parts = result_latest.split("/")[1:-2]
    caption = " ".join(result_latest_parts).upper()

    df = pd.read_json(result_latest, lines=True)

    top_n = 5
    df = df.truncate(after=top_n - 1, axis=0)
    df.index += 1
    df = (
        df.style.format(precision=5)
        .format_index(str.upper, axis=1)
        .set_properties(**{"text-align": "left"})
        .set_caption(caption)
    )
    display(df)

In [ ]:
!pip install locust
!which locust

## Locust Testing

If the configuration file includes `test` object, we can do [Locust](https://locust.io/) based throughput testing, as shown below. Next cell will run until testing is complete. The duration of the testing is specified in  `os.environ["RUN_TIME"]` below, which by default is 2 minutes. The `os.environ["SPAWN_RATE"]` specifies ramp up rate for the Locust Users. 

In [ ]:
import subprocess
import time

ts = round(time.time() * 1000)
if test_spec:
    os.environ["STREAMING_ENABLED"] = str(streaming_enabled)
    os.environ["CONTENT_TYPE"] = "application/json"
    os.environ["ENDPOINT_NAME"] = (
        f"https://runtime.sagemaker.{aws_region}.amazonaws.com/endpoints/{endpoint_name}/invocations"
    )
    os.environ["USERS"] = str(test_spec.get("locust_users", 4))
    os.environ["WORKERS"] = str(test_spec.get("locust_workers", 1))
    os.environ["RUN_TIME"] = "2m"
    os.environ["SPAWN_RATE"] = str(test_spec.get("locust_users", 4))
    os.environ["SCRIPT"] = "endpoint_user.py"
    results_locust_path = os.path.join(
        output_dir,
        "locust-testing",
        sm_model_name,
        f"type={instance_type}",
        f"count={instance_count}",
        f"slen={seq_len}",
        f"tp={tp_degree}",
        f"rbs={rbs}",
    )
    os.environ["RESULTS_PREFIX"] = f"{results_locust_path}/results-{ts}"
    os.environ["TASK_NAME"] = test_spec.get("task_name", "text-generation")

    try:
        with open("run_locust.log", "w") as logfile:
            print(f"Start Locust testing; logfile: run_locust.log; results: {results_locust_path}")
            path = os.path.join(os.getcwd(), "run_locust.sh")
            os.chmod(path, stat.S_IRUSR | stat.S_IEXEC)
            process = subprocess.Popen(
                path, encoding="utf-8", shell=True, stdout=logfile, stderr=subprocess.STDOUT
            )
            process.wait()
            logfile.flush()
            print(f"Locust testing completed")

        print(f"Uploading locust results to S3 bucket...")
        for root, dirs, files in os.walk(results_locust_path):
            for file in files:
                full_path = os.path.join(root, file)
                with open(full_path, "rb") as data:
                    s3_client.upload_fileobj(data, s3_bucket, full_path)
        print(f"Uploading locust results to S3 bucket completed.")
    except Exception as e:
        print(f"exception occurred: {e}")

## Visualize Locust Results

Below we first visualize the results of the Locust testing in a tabel. 

In [ ]:
import pandas as pd
from IPython.display import display
import numpy as np

if test_spec:
    results_path = os.environ["RESULTS_PREFIX"] + "_stats.csv"
    df = pd.read_csv(results_path)
    df = df.replace(np.nan, "")

    top_n = 1
    caption = f"{endpoint_name} type={instance_type} count={instance_count} tp={tp_degree} slen={seq_len} rbs={rbs}".upper()
    df = df.truncate(after=top_n - 1, axis=0)
    df = df.style.format(precision=6).set_properties(**{"text-align": "left"}).set_caption(caption)
    display(df)

Next, we visualize the key metrics in a plot.

In [ ]:
if test_spec:
    results_path = os.environ["RESULTS_PREFIX"] + "_stats.csv"
    df = pd.read_csv(results_path)
    df = df[
        [
            "Median Response Time",
            "Average Response Time",
            "Min Response Time",
            "Max Response Time",
            "95%",
        ]
    ]
    df = df.replace(np.nan, "")

    top_n = 1
    df = df.truncate(after=top_n - 1, axis=0)
    df.plot.bar()

## Cleanup

This concludes the notebook. Below. we delete the **deployed** SageMaker endpoint, endpoint configuration, and the model. 

In [ ]:
response = sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Delete Endpoint response: {response}")

response = sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Delete Endpoint Config response: {response}")

response = sm_client.delete_model(ModelName=sm_model_name)
print(f"Delete Model response: {response}")

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/advanced_functionality|distributed_tensorflow_mask_rcnn|mask-rcnn-scriptmode-fsx.ipynb)
